In [68]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import  WikipediaAPIWrapper
from langchain.tools import DuckDuckGoSearchResults


llm = ChatOpenAI(temperature=0.1)

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the xz backdoor"
    )
class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to search for information on the xz backdoor.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema


    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        results= wiki.run(query)
        with open('output.txt', 'w') as file:
            file.write("Wikipedia Results:\n" + results + "\n\n")
        return results

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to search for information on the xz backdoor.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        results = ddg.run(query)
        with open('output.txt', 'a') as file:
            file.write("DuckDuckGo Results:\n" + results + "\n\n")
        return results


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool()
    ],
)

prompt = "Research about the xz backdoor in wikipedia and duckduckgo sites"

output = agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'xz backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless data compresso

In [76]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.utilities import  WikipediaAPIWrapper


def get_wikipedia(inputs):
    wiki = WikipediaAPIWrapper()
    
    query = inputs["query"]
    
    return wiki.run(f"research of  {query}")



def get_duckduckgo(inputs):
    query= inputs["query"]
    # ddg =  DuckDuckGoSearchResults()
    ddg=DuckDuckGoSearchAPIWrapper()
    return ddg.run(f"research of  {query}")


functions_map = {
    "get_wikipedia": get_wikipedia,
    "get_duckduckgo": get_duckduckgo,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_wikipedia",
            "description": "Given the query return the results of searching the Wikipedia site",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "query to search for",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_duckduckgo",
            "description": "Given the query return the results of searching the DuckDuckGoSearchTo site",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "query to search for",
                    },
                },
                "required": ["query"],
            },
        },
    },
   
]




In [95]:
import openai as client
assistant = client.beta.assistants.create(
  name="Search Assistant",
  instructions="You help users do research information. It takes a query as an argument.",
  model="gpt-4-1106-preview",
  tools = functions,
)
assistant


Assistant(id='asst_KNt1YYI4grEcjV7D8kcryRDY', created_at=1713110994, description=None, file_ids=[], instructions='You help users do research information. It takes a query as an argument.', metadata={}, model='gpt-4-1106-preview', name='Search Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipedia', description='Given the query return the results of searching the Wikipedia site', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'query to search for'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_duckduckgo', description='Given the query return the results of searching the DuckDuckGoSearchTo site', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'query to search for'}}, 'required': ['query']}), type='function')])

In [100]:
thread=client.beta.threads.create(
  messages=[
    {
      "role":"user",
      "content":"Research about the xz backdoor"
    }
  ]
)
thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
run


Run(id='run_lGiErJScXiisHWUySKD8PYmk', assistant_id='asst_KNt1YYI4grEcjV7D8kcryRDY', cancelled_at=None, completed_at=None, created_at=1713111099, expires_at=1713111699, failed_at=None, file_ids=[], instructions='You help users do research information. It takes a query as an argument.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_UOxzofc4U6KE1IY6wlxQCE70', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipedia', description='Given the query return the results of searching the Wikipedia site', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'query to search for'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_duckduckgo', description='Given the query return the results of searching the DuckDuckGoSearchTo site', parameters={'type': 'object', 'properties': {'query': {'type': 'string

In [97]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
run

In [78]:
import json
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

In [101]:
get_run(run.id, thread.id).status

'requires_action'

In [85]:
def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    print("*****", run.required_action.submit_tool_outputs.tool_calls, "\n\n")
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outputs
    )

In [93]:
def get_run(run_id, thread_id):
  return client.beta.threads.runs.retrieve(
    run_id=run_id,
    thread_id=thread_id
  )

def send_message(thread_id, content):
  return client.beta.threads.messages.create(
    thread_id=thread_id, role="user", content=content
  )

def get_messages(thread_id):
  messages = client.beta.threads.messages.list(
    thread_id=thread_id
  )

  messages = list(messages)
  print(messages)
  for message in messages:
    print(f"{message.role}:{message.content[0].text.value}")

  print("===========================================")
  messages.reverse()
  print(messages)
  for message in messages:
    print(f"{message.role}:{message.content[0].text.value}")


In [102]:
# get_run(run.id, thread.id).status
get_messages(thread.id)

AttributeError: 'SyncCursorPage[Message]' object has no attribute 'reverse'

In [91]:
get_run(run.id, thread.id).status

'completed'

In [90]:
submit_tool_outputs(run.id, thread.id)

***** [RequiredActionFunctionToolCall(id='call_yBu7w6RTMiqwFPhXw6LwTri6', function=Function(arguments='{"query": "xz backdoor"}', name='get_wikipedia'), type='function'), RequiredActionFunctionToolCall(id='call_GNTOkuyqCDOGoBdnsqbWOVbU', function=Function(arguments='{"query": "xz backdoor"}', name='get_duckduckgo'), type='function')] 


Calling function: get_wikipedia with arg {"query": "xz backdoor"}
Calling function: get_duckduckgo with arg {"query": "xz backdoor"}


Run(id='run_pZF0BavupJ8Fpxajwd4e6fAf', assistant_id='asst_yoTz0fGp7N1679kWkdN2nFhy', cancelled_at=None, completed_at=None, created_at=1713103494, expires_at=1713104094, failed_at=None, file_ids=[], instructions='You help users do research information. It takes a query as an argument.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713103494, status='queued', thread_id='thread_Lx9tGC3fIkerxxRL1N4z3q5P', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipedia', description='Given the query return the results of searching the Wikipedia site', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'query to search for'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_duckduckgo', description='Given the query return the results of searching the DuckDuckGoSearchTo site', parameters={'type': 'object', 'properties': {'query': {'type': '

In [92]:
get_messages(thread.id)

[Message(id='msg_Px4g0VrrkdiQxh1xXP0H4B99', assistant_id='asst_yoTz0fGp7N1679kWkdN2nFhy', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The xz backdoor appears to be a recently discovered security issue affecting the Linux utility xz, specifically within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024. The backdoor enables remote code execution by an attacker with a particular Ed448 private key. The affected versions of xz had not yet been widely deployed to production systems by the time of discovery but were present in the development versions of major Linux distributions.\n\nThis backdoor was reported by software developer Andres Freund on March 29, 2024, earning the designation of CVE-2024-3094 with a CVSS score of 10.0, indicating the highest severity level.\n\nThe backdoor operates by manipulating the `sshd` (OpenSSH Daemon), which is responsible for secure internet connections via the SSH protocol. This manipulation allow

In [104]:
user_query="Rearch xz backkdoor"
assistant = client.beta.assistants.create(
            name="Search Assistant",
            instructions="You help users do research information. It takes a query as an argument.",
            model="gpt-4-1106-preview",
            tools=functions,
        )
assistant_id=assistant.id

thread = client.beta.threads.create(
    messages=[{"role": "user", "content": f"Research about {user_query}"}]
)

run = client.beta.threads.runs.create(
    thread_id=thread.id, assistant_id=assistant_id
)


In [105]:
submit_tool_outputs(run.id, thread.id)

***** [RequiredActionFunctionToolCall(id='call_lNEnTqGFPquXjKtfc9kTzotY', function=Function(arguments='{"query": "xz backdoor"}', name='get_wikipedia'), type='function'), RequiredActionFunctionToolCall(id='call_WlD0SZHCoXqsvD2d2MZP2LLL', function=Function(arguments='{"query": "xz backdoor"}', name='get_duckduckgo'), type='function')] 


Calling function: get_wikipedia with arg {"query": "xz backdoor"}
Calling function: get_duckduckgo with arg {"query": "xz backdoor"}


Run(id='run_Cl3fv5GUY7GgMFV9GbDldnDV', assistant_id='asst_JqEyHMBYvzDJW91LUr7IqPcM', cancelled_at=None, completed_at=None, created_at=1713117168, expires_at=1713117768, failed_at=None, file_ids=[], instructions='You help users do research information. It takes a query as an argument.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713117168, status='queued', thread_id='thread_ecL0VBp5lksPyMhpJ2W6jBak', tools=[FunctionTool(function=FunctionDefinition(name='get_wikipedia', description='Given the query return the results of searching the Wikipedia site', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'query to search for'}}, 'required': ['query']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_duckduckgo', description='Given the query return the results of searching the DuckDuckGoSearchTo site', parameters={'type': 'object', 'properties': {'query': {'type': '